# 커피지수 part_2

In [1]:
import numpy as np
import pandas as pd

In [3]:
starbucks = pd.read_csv('./data1/스타벅스.csv')
coffeebean = pd.read_csv('./data1/커피빈.csv')
ediya = pd.read_csv('./data1/이디야.csv')
paik = pd.read_csv('./data1/빽다방.csv')


In [18]:
# 고성군 - 공성(강원), 고성(경남)
# 광역시 - 서울 용산, 서울 서대문, 대전 서구, 대전 유성, 세종
# 행정구 - 수원 장안, 용인 수지, 고양 일산동, 창원 합포, 창원 회원
# id 를 추가해서 카토그램이랑 연동할수 있게... 뭐 그런거다
# 함수로 만들거다 - 조건문
tmp_gu_dict = {
    '수원':['장안구','권선구', '팔달구', '영통구'],
    '성남':['수정구','중원구', '분당구'],
    '안양':['만안구', '동안구'],
    '안산':['상록구', '단원구'],
    '고양':['덕양구', '일산동구', '일산서구'],
    '용인':['처인구', '기흥구', '수지구'],
    '청주':['상당구', '서원구', '흥덕구', '청원구'],
    '천안':['동남구', '서북구'],
    '전주':['완산구', '덕진구'],
    '포항':['남구', '북구'],
    '창원':['의창구', '성산구', '진해구', '마산합포구', '마산회원구']
    }

In [34]:
coffeebean.head()

,상호명,시도명,시군구명,ID
0,커피빈,강원도,동해시,동해
1,커피빈,강원도,원주시,원주
2,베가커피빈스,경기도,수원시 장안구,수원 장안
3,커피빈,경기도,김포시,김포
4,커피빈,경기도,부천시,부천


In [19]:
#df= pd.concat([starbucks, coffeebean, ediya, paik])

ediya[ediya.시군구명.str.contains('고성')]

,상호명,시도명,시군구명
668,이디야커피,경상남도,고성군


In [20]:
starbucks[starbucks.시군구명.str.contains('고성')]

,상호명,시도명,시군구명
34,스타벅스,강원도,고성군


In [21]:
metro_list = ['서울특별시', '부산광역시', '대구광역시', '인천광역시', '대전광역시', '광주광역시', '울산광역시']

In [22]:
def get_ID(df):
    si_name = [None] * len(df)
    for i in df.index:
        if df.시도명[i] in metro_list:
            if len(df.시군구명[i]) == 2:
                #서울 중구, 대전 서구
                si_name[i] = df.시도명[i][:2] + ' ' + df.시군구명[i]
            else:# 서울 용산, 서울 서대문
                si_name[i] = df.시도명[i][:2] + ' ' + df.시군구명[i][:-1]
        else:
            si_len = len(df.시군구명[i].split())
            if si_len == 1:
                if df.시군구명[i][:-1] == '고성':
                    if df.시도명[i] == '강원도':
                        si_name[i] = '고성(강원)'
                    else:
                        si_name[i] = '고성(경남)'
                elif df.시군구명[i].find('세종') == 0:
                    si_name[i] = '세종'
                else:
                    si_name[i] = df.시군구명[i][:-1] #세종, 광명, 김포, 강릉
            else:
                admin_gu = df.시군구명[i].split()[-1]
                for key, value in tmp_gu_dict.items():
                    if admin_gu in value:
                        if len(admin_gu) == 2:
                            si_name[i] = key + ' ' + admin_gu
                        elif len(admin_gu) == 5: # 창원 합포, 창원 화원
                            si_name[i] = key + ' ' + admin_gu[2:-1]
                        else: #수원 팔달, 용인 지
                            si_name[i] = key + ' ' + admin_gu[:-1]
                        
    return si_name


In [25]:
starbucks['ID'] = get_ID(starbucks)
starbucks.head(10)

,상호명,시도명,시군구명,ID
0,스타벅스강릉안목항점,강원도,강릉시,강릉
1,스타벅스춘천후평DT점,강원도,춘천시,춘천
2,스타벅스,강원도,춘천시,춘천
3,스타벅스설악워터피아점,강원도,속초시,속초
4,스타벅스,강원도,원주시,원주
5,스타벅스,강원도,춘천시,춘천
6,스타벅스,강원도,춘천시,춘천
7,스타벅스,강원도,속초시,속초
8,스타벅스,강원도,원주시,원주
9,스타벅스,강원도,강릉시,강릉


In [33]:
# 각 ID 별 개수 

# 스벅: 191
# 커피빈: 92
# 이디야: 238
# 빽다방: 174

starbucks['ID'] = get_ID(starbucks)
coffeebean['ID'] = get_ID(coffeebean)
ediya['ID'] = get_ID(ediya)
paik['ID'] = get_ID(paik)

len(starbucks.ID.unique()), len(coffeebean.ID.unique()), len(ediya.ID.unique()), len(paik.ID.unique())

(191, 92, 238, 174)

In [36]:
# 모든 단위 도시를 합하면 합집합

city_set = set(starbucks.ID.unique()) | set(coffeebean.ID.unique()) | set(ediya.ID.unique()) | set(paik.ID.unique())
len(city_set)

244

In [37]:
# starbucks만 있고, ediya / piak 가 없는곳
set(starbucks.ID.unique()) - set(ediya.ID.unique()) - set(paik.ID.unique())

{'계룡', '고성(강원)', '영양', '진도'}

In [38]:
# 브랜드별 매장수 계산
# 스벅

starbucks.groupby('ID')[['상호명']].count().head()

,상호명
ID,
가평,2
강릉,4
거제,6
경산,8
경주,7


In [39]:
pt_sb = starbucks.groupby('ID')[['상호명']].count()
pt_sb.reset_index(inplace=True)
pt_sb.rename(columns={'상호명':'스타벅스'}, inplace=True)
pt_sb.head()

,ID,스타벅스
0,가평,2
1,강릉,4
2,거제,6
3,경산,8
4,경주,7


In [40]:
pt_cb = coffeebean.groupby('ID')[['상호명']].count()
pt_cb.reset_index(inplace=True)
pt_cb.rename(columns={'상호명':'커피빈'}, inplace=True)
pt_cb.head()

,ID,커피빈
0,경주,2
1,고양 덕양,1
2,고양 일산동,1
3,고양 일산서,1
4,광명,1


In [41]:
pt_ed = ediya.groupby('ID')[['상호명']].count()
pt_ed.reset_index(inplace=True)
pt_ed.rename(columns={'상호명':'이디야'}, inplace=True)
pt_ed.head()

,ID,이디야
0,가평,9
1,강릉,17
2,강진,1
3,거제,9
4,거창,1


In [46]:
pt_bd = paik.groupby('ID')[['상호명']].count()
pt_bd.reset_index(inplace=True)
pt_bd.rename(columns={'상호명':'빽다방'}, inplace=True)
pt_bd.head()

,ID,빽다방
0,강릉,2
1,거창,1
2,경산,4
3,경주,1
4,고령,1


In [48]:
len(pt_sb), len(pt_cb), len(pt_ed), len(pt_bd)

(191, 92, 238, 174)

In [49]:
# merge로 커피데이터 병합
cf = pd.merge(pt_ed, pt_sb, how='left')
cf.head()

,ID,이디야,스타벅스
0,가평,9,2.0
1,강릉,17,4.0
2,강진,1,NaN
3,거제,9,6.0
4,거창,1,NaN


In [50]:
cf = pd.merge(cf, pt_cb, how='left')  # outer, inner 이런거로도 연습해봐이자스가
cf = pd.merge(cf, pt_bd, how='left')
cf.head()

,ID,이디야,스타벅스,커피빈,빽다방
0,가평,9,2.0,NaN,NaN
1,강릉,17,4.0,NaN,2.0
2,강진,1,NaN,NaN,NaN
3,거제,9,6.0,NaN,NaN
4,거창,1,NaN,NaN,1.0


In [52]:
#NA값 0으로 
cf.fillna(0, inplace=True)
cf.스타벅스 = cf.스타벅스.astype(int)
cf.커피빈 = cf.커피빈.astype(int)
cf.이디야 = cf.이디야.astype(int)
cf.빽다방 = cf.빽다방.astype(int)

In [57]:
#cf.dtypes int로 바뀐지 확인
cf.head()

,ID,이디야,스타벅스,커피빈,빽다방
0,가평,9,2,0,0
1,강릉,17,4,0,2
2,강진,1,0,0,0
3,거제,9,6,0,0
4,거창,1,0,0,1


In [68]:
cf['커피지수'] = (cf.스타벅스+cf.커피빈)/(cf.이디야+cf.빽다방)
cf.head()
cf.to_csv('./data1/커피지수.csv', index=False) # 이렇게 저장해라~!

In [60]:
df1 = pd.DataFrame({
    '고객번호': [1001, 1002, 1003, 1004, 1005, 1006, 1007],
    '이름': ['둘리', '도우너', '또치', '길동', '희동',
          '마이콜', '영희']
}, columns=['고객번호', '이름'])
df1

,고객번호,이름
0,1001,둘리
1,1002,도우너
2,1003,또치
3,1004,길동
4,1005,희동
5,1006,마이콜
6,1007,영희


In [62]:
df2 = pd.DataFrame({
    '고객번호': [1001, 1001, 1003, 1005, 1008, 1001],
    '금액': [10000, 20000, 15000, 5000, 100000, 30000]
}, columns=['고객번호', '금액'])
df2

,고객번호,금액
0,1001,10000
1,1001,20000
2,1003,15000
3,1005,5000
4,1008,100000
5,1001,30000


In [67]:
# default how='inner'
pd.merge(df1, df2, how='outer')

,고객번호,이름,금액
0,1001,둘리,10000.0
1,1001,둘리,20000.0
2,1001,둘리,30000.0
3,1002,도우너,NaN
4,1003,또치,15000.0
5,1004,길동,NaN
6,1005,희동,5000.0
7,1006,마이콜,NaN
8,1007,영희,NaN
9,1008,NaN,100000.0
